### 문제 6-1 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용하기)


In [18]:
from dotenv import load_dotenv
import re
from textwrap import dedent
import warnings
warnings.filterwarnings("ignore")
from langchain_community.vectorstores import FAISS
from langchain_ollama  import OllamaEmbeddings
from langchain_core.tools import tool
from typing import List
from langchain_community.tools import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage
from textwrap import dedent
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from typing import List
from langchain_core.documents import Document

load_dotenv()

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# Tool 정의 
@tool
def search_menu(query: str) -> List[str]:
    """카페 메뉴에서 정보를 검색합니다."""
    docs = menu_db.similarity_search(query, k=6)

    formatted_docs = "\n\n---\n\n".join(
        [
            f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )

    if len(docs) > 0:
        return formatted_docs
    
    return "관련 메뉴 정보를 찾을 수 없습니다."

# Tool 정의 
@tool
def search_web(query: str) -> List[str]:
    """데이터베이스에 존재하지 않는 정보 또는 최신 정보를 인터넷에서 검색합니다."""

    tavily_search = TavilySearchResults(max_results=3)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n\n---\n\n".join(
        [
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in docs
        ]
    )

    if len(docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."


# LLM 모델 
# llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.3
)

# 도구 목록
tools = [search_menu, search_web]
llm_with_tools = llm.bind_tools(tools=tools)

system_prompt = dedent("""
당신은 사용자 질문에 답변하는 카페 AI 어시스턴트입니다.  
제공된 도구들을 활용해 정확한 정보를 전달해야 합니다.

[작동 원칙]
1. 질문 이해: 사용자의 질문을 정확히 파악하세요
2. 도구 활용: 필요한 정보는 반드시 제공된 도구로 조회
3. 출처 명시: 도구 사용 후 즉시 아래 형식으로 출처 표기
4. 답변 구성: 도구 결과와 출처를 포함해 명확히 답변
5. 완성 판단: 충분한 정보가 모이면 최종 답변 제공

[도구 사용 형식]
액션: 도구_이름  
액션 입력: 도구에_넘길_입력값  

[출처 표기 형식]
[출처: 도구_이름 | 문서_제목/항목명 | URL/파일경로]

[예시 1 - 메뉴 검색]
액션: search_menu  
액션 입력: 아메리카노  

(도구 실행 후)  
[출처: search_menu | 아메리카노 | ../data/cafe_menu_data.txt]  
아메리카노 정보: 아메리카노 4,500원...

[예시 2 - 웹 검색]  
액션: search_web  
액션 입력: AI 역사  

(도구 실행 후)  
[출처: search_web | AI 역사 | https://ko.wikipedia.org/wiki/인공지능]  
AI 역사는 1950년대부터...

[주의사항]
1. 도구가 필요없는 질문은 직접 답변
2. 모든 사실 정보는 반드시 출처 동반
3. 출처 없이는 어떠한 정보도 제공하지 말 것
4. 최종 답변은 질문과 직접 관련된 명확한 내용으로 구성
""")

def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    
    # 정규표현식으로 가격, 설명 등 추출
    price_match = re.search(r'₩([\d,]+)', content)
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL)
    
    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description_match.group(1).strip() if description_match else "설명 없음"
    }
class GraphState(TypedDict):
    messages: List[AIMessage | HumanMessage]
    query_type: str 
    
def classify_query(state: GraphState) -> GraphState:
    last_message = state["messages"][-1].content.lower()
    qtype = "기타"
    if "메뉴" in last_message or "뭐가 있나요" in last_message or "종류" in last_message:
        qtype = "메뉴"
    elif "가격" in last_message or "얼마" in last_message:
        qtype = "가격"
    elif "추천" in last_message or "인기 메뉴" in last_message or "어떤" in last_message:
        qtype = "추천"
    return {"query_type": qtype}
    
def handle_menu_query(state: GraphState) -> GraphState:
    user_message = state["messages"][-1].content
    print(f"사용자 메시지: {user_message}")

    docs = menu_db.similarity_search(user_message, k=4)
    print(f"검색된 문서 수: {len(docs)}")

    if not docs:
        response_content = "요청하신 메뉴 정보를 찾을 수 없습니다."
        return {"messages": [AIMessage(content=response_content)]}

    menu_infos = [extract_menu_info(doc) for doc in docs]
    
    response_parts = ["저희 카페 메뉴는 다음과 같습니다:"]
    for info in menu_infos:
        response_parts.append(
            f"- **{info['name']}**: {info['description']} (가격: {info['price']})"
        )
    
    response_content = "\n".join(response_parts)
    return {"messages": [AIMessage(content=response_content)]}


def handle_price_query(state: GraphState) -> GraphState:
    user_message = state["messages"][-1].content
    
    docs = menu_db.similarity_search(user_message + " 가격", k=5) 

    if not docs:
        response_content = "요청하신 메뉴의 가격 정보를 찾을 수 없습니다."
        return {"messages": [AIMessage(content=response_content)]}

    menu_infos = [extract_menu_info(doc) for doc in docs]
    
    response_parts = ["다음은 요청하신 메뉴의 가격 정보입니다:"]
    for info in menu_infos:
        if info['price'] != "가격 정보 없음":
            response_parts.append(f"-{info['name']}: {info['price']}")
        else:
            response_parts.append(f"-{info['name']}: 가격 정보를 찾을 수 없습니다.")
    
    response_content = "\n".join(response_parts)
    return {"messages": [AIMessage(content=response_content)]}

def handle_recommendation_query(state: GraphState) -> GraphState:
    user_message = state["messages"][-1].content

    docs = menu_db.similarity_search(user_message, k=3)
    
    if not docs:
        response_content = "죄송합니다. 추천해 드릴 만한 메뉴 정보를 찾을 수 없습니다."
        return {"messages": [AIMessage(content=response_content)]}

    menu_infos = [extract_menu_info(doc) for doc in docs]
    
    response_parts = ["고객님께 다음 메뉴를 추천해 드립니다:"]
    for info in menu_infos:
        response_parts.append(
            f"- **{info['name']}**: {info['description']} (가격: {info['price']})"
        )
    
    response_content = "\n".join(response_parts)
    return {"messages": [AIMessage(content=response_content)]}

def handle_other_query(state: GraphState) -> GraphState:
    user_message = state["messages"][-1].content
    messages_for_llm = [SystemMessage(content=system_prompt), HumanMessage(content=user_message)]
    response = llm_with_tools.invoke(messages_for_llm)
    return {"messages": [response]}

builder = StateGraph(GraphState)

builder.add_node("classify", classify_query)
builder.add_node("menu_handler", handle_menu_query)
builder.add_node("price_handler", handle_price_query)
builder.add_node("recommendation_handler", handle_recommendation_query)
builder.add_node("other_handler", handle_other_query)

# 엣지 정의
builder.add_edge(START, "classify") 

# 조건부 엣지: 문의 유형에 따라 다른 핸들러로 라우팅
builder.add_conditional_edges(
    "classify", 
    lambda state: state['query_type'],
    {
        "메뉴": "menu_handler",
        "가격": "price_handler",
        "추천": "recommendation_handler",
        "기타": "other_handler", # 기타 문의는 일반 LLM 처리기로 보냄
    },
)

builder.add_edge("menu_handler", END)
builder.add_edge("price_handler", END)
builder.add_edge("recommendation_handler", END)
builder.add_edge("other_handler", END)

graph = builder.compile()


In [20]:
print("--- 테스트 시작 ---")

# 메뉴 문의 테스트
inputs_menu = {"messages": [HumanMessage(content="아메리카노 메뉴는 무엇인가요?")]}
result_menu = graph.invoke(inputs_menu)
print("\n--- 메뉴 문의 결과 ---")
for m in result_menu['messages']:
    print(m.content)

# 가격 문의 테스트
inputs_price = {"messages": [HumanMessage(content="아메리카노 가격이 얼마인가요?")]}
result_price = graph.invoke(inputs_price)
print("\n--- 가격 문의 결과 ---")
for m in result_price['messages']:
    print(m.content)

# 추천 요청 테스트
inputs_recommend = {"messages": [HumanMessage(content="오늘의 추천 메뉴가 있나요?")]}
result_recommend = graph.invoke(inputs_recommend)
print("\n--- 추천 요청 결과 ---")
for m in result_recommend['messages']:
    print(m.content)

# 기타 문의 테스트 (LLM이 처리)
inputs_other = {"messages": [HumanMessage(content="카페 운영 시간은 어떻게 되나요?")]}
result_other = graph.invoke(inputs_other)
print("\n--- 기타 문의 결과 ---")
for m in result_other['messages']:
    print(m.content)

inputs_seq = {"messages": [HumanMessage(content="카푸치노 가격은요?")]}
result_seq = graph.invoke(inputs_seq)
print("\n--- 연속적인 대화 (카푸치노 가격) 결과 ---")
for m in result_seq['messages']:
    print(m.content)


--- 테스트 시작 ---
사용자 메시지: 아메리카노 메뉴는 무엇인가요?
검색된 문서 수: 4

--- 메뉴 문의 결과 ---
저희 카페 메뉴는 다음과 같습니다:
- **아메리카노**: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다. (가격: ₩4,500)
- **아이스 아메리카노**: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다. (가격: ₩4,500)
- **카푸치노**: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다. (가격: ₩5,000)
- **프라푸치노**: 에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료입니다. 부드럽고 크리미한 질감이 특징이며, 휘핑크림을 올려 달콤함을 더했습니다. 여름철 인기 메뉴입니다. (가격: ₩7,000)

--- 가격 문의 결과 ---
다음은 요청하신 메뉴의 가격 정보입니다:
-아메리카노: ₩4,500
-아이스 아메리카노: ₩4,500
-카푸치노: ₩5,000
-프라푸치노: ₩7,000
-카라멜 마키아토: ₩6,500
사용자 메시지: 오늘의 추천 메뉴가 있나요?
검색된 문서 수: 4

--- 추천 요청 결과 ---
저희 카페 메뉴는 다음과 같습니다:
- **바닐라 라떼**: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다. (가격: ₩6,000)
- **티라미수**: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 